<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/philipp/notebooks/Dashboard_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q&A Dasbboard / Live Demo

In [1]:
%%capture
!pip install gradio
!pip install git+https://github.com/openai/whisper.git

import gradio as gr
import whisper

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-sv9zvgqi
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-sv9zvgqi
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Load the Whisper model
model = whisper.load_model("base")

# Evaluation function for audio input using Whisper
def evaluate_answer(audio_filepath, current_question):
    """
    Transcribes and evaluates the user's spoken answer for the current question.
    """
    try:
        # Transcribe the audio using Whisper
        result = model.transcribe(audio_filepath)
        transcription = result['text']

        # Simulate evaluation logic (placeholder)
        if "skip" in transcription.lower():
            return False, f"Could not understand the answer to: '{current_question}'"
        return True, transcription
    except Exception as e:
        return False, f"Error during transcription: {str(e)}"

# Questions and metadata
questions = [
    {"question": "What is your name?", "attempts": 0, "answered": False, "answer": None},
    {"question": "What is your favorite food?", "attempts": 0, "answered": False, "answer": None},
    {"question": "What is your favorite hobby?", "attempts": 0, "answered": False, "answer": None},
]

# Maximum allowed attempts per question
MAX_ATTEMPTS = 3

def process_audio(audio_filepath):
    """
    Handles audio input, evaluates the answer, updates questions, and determines the next step.
    """
    global questions

    # Find the next unanswered question
    remaining_questions = [q for q in questions if not q["answered"] and q["attempts"] < MAX_ATTEMPTS]
    if not remaining_questions:
        # All questions answered or skipped
        summary = "\n".join(
            [f"{q['question']}: {q['answer'] if q['answered'] else 'Skipped'}" for q in questions]
        )
        return "All questions completed!", "", summary

    current_question = remaining_questions[0]

    # Evaluate the current question
    success, evaluation_result = evaluate_answer(audio_filepath, current_question["question"])
    current_question["attempts"] += 1

    if success:
        current_question["answered"] = True
        current_question["answer"] = evaluation_result
        message = f"Answer recorded: {evaluation_result}"
    else:
        message = f"Attempt {current_question['attempts']} failed: {evaluation_result}"

    # Check if the question is now skipped due to exceeding attempts
    if current_question["attempts"] >= MAX_ATTEMPTS and not current_question["answered"]:
        message += f" | Question '{current_question['question']}' skipped after {MAX_ATTEMPTS} attempts."

    # Determine the next question
    remaining_questions = [q for q in questions if not q["answered"] and q["attempts"] < MAX_ATTEMPTS]
    next_question = remaining_questions[0]["question"] if remaining_questions else "All questions completed!"

    return message, next_question, ""

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## Voice-Based Interactive Questionnaire")

    question_display = gr.Textbox(label="Current Question", interactive=False, value=questions[0]["question"])
    audio_input = gr.Audio(type="filepath", label="Answer via voice input")
    output = gr.Textbox(label="Message")
    summary_display = gr.Textbox(label="Summary", interactive=False)

    audio_input.change(
        process_audio,
        inputs=[audio_input],
        outputs=[output, question_display, summary_display],
    )

demo.launch()


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 72.4MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c5da31b7e1c051e4ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# NEU
Source: https://colab.research.google.com/github/fastforwardlabs/whisper-openai/blob/master/WhisperDemo.ipynb#scrollTo=wIRFnTn3Fzua

1. Select questionnaire als Dropdown

In [5]:
%%capture

! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

!apt install ffmpeg
!apt-get install libportaudio2

import os
import numpy as np
import tensorflow
import torch
import pandas as pd
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from google.colab import output
output.enable_custom_widget_manager()

In [7]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [8]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav my_recording.wav -y -hide_banner -loglevel panic

model = whisper.load_model("base.en")

options = whisper.DecodingOptions(language="english", task="transcribe", without_timestamps=True)

audio = whisper.load_audio("my_recording.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)
result = model.decode(mel, options)

result.text

100%|███████████████████████████████████████| 139M/139M [00:08<00:00, 17.7MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

'Yes, I would like to answer the question of how big my company is. My company has about 51 to 200 workers, employees and yeah, the topic my company is very satisfied with your thing.'